## ChatGPT with Few Shot

Include some examples for ChatGPT of histograms.

Docs:

* how to upload images: https://platform.openai.com/docs/guides/vision
* types of models: https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
* token limits: https://platform.openai.com/settings/organization/limits
* models: https://platform.openai.com/docs/models/model-endpoint-compatibility

In [1]:
#!pip install openai

In [2]:
#dir_api = '/Users/jnaiman/Dropbox/Paper_JCDL2025/chatgpt_api/' # where to store API results

dir_api = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/' #store API results for example-hists

key_file = '/Users/jnaiman/.openai/key.txt'

# for fewshot
fewshot_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/fewshot/' # assume images in "imgs", jsons in "jsons"

# for testing images
jsons_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/' # directory where jsons created with figure are stored
imgs_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/imgs/' # where images are stored

# for saving temp images for reading in
tmp_dir = '/Users/jnaiman/Downloads/tmp/'

img_format = 'jpeg'

In [3]:
import openai
import base64
from openai import OpenAI
from PIL import Image
import numpy as np
import json
import re
import pickle
import os
from glob import glob
from copy import deepcopy

# debug
from importlib import reload

from sys import path
path.append('../')
import utils.llm_utils
reload(utils.llm_utils)
from utils.llm_utils import parse_qa, load_image, get_img_json_pair, parse_for_errors

In [4]:
def make_question_and_prompt(question_list, encoded_image, verbose = True):
    # current question format is: ['persona', 'context','question', 'format'] (see readme in example_hists)
    question = question_list['context'] + " " + question_list['question'] + " " + question_list['format']
    # lowercase the first word, just in case
    question = question.lstrip() # no whitespace
    question = question[0].lower() + question[1:]
    if verbose: print('   on question:',question)
    # Prepare the API request
    prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
    prompt_save = f"I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, {question}"
    return prompt, prompt_save, question

In [5]:
def create_fewshot(fewshot_dir, question_list, 
                   tmp_dir = '/Users/jnaiman/Downloads/tmp/',
                   img_dir = 'imgs', json_dir = 'jsons',
                   fac = 1.0, restart=False, verbose=False, 
                    types = ['(words + list)', '(words)']
):
    # get question
    #question = 
    prompt = 'Here are some example questions with figures and their respective answers. Reference examples delimited with “”"\n'
    prompt += '"""\n'
    prompt_dummy = deepcopy(prompt)
    json_files = glob(fewshot_dir + '/' + json_dir + '/*')
    # print(fewshot_dir + '/' + json_dir + '/*')
    # print(json_files)
    if verbose:
        print('MAIN QUESTION:', question_list['Q'])
    for json_path in json_files:
        if verbose: 
            print('-------- JSON: ' + json_path, '-----------------')
        img_path = imgs_dir + json_path.split('/')[-1].removesuffix('.json') + '.' + img_format
        encoded_image, img_format_media, base_json, err = get_img_json_pair(img_path, json_path, None, 
                                                            fac=fac, restart=restart,
                                                            tmp_dir=tmp_dir)
        
        if err:
            print('[ERROR]: error in create_fewshot')
            print('json_path:', json_path)
            print('dir_api:', dir_api)
            continue
        # response, prompt_dummy_full = send_to_chatgpt(question_list, client, img_path, encoded_image,
        #             model = model, img_format = img_format_media,
        #             test_run = True, subset_questions_by_keys=subset_questions_by_keys)
        _, prompt_dummy_full, _ =  make_question_and_prompt(question_list, encoded_image, verbose = verbose)

        #**HERE** response['a'] needs to be from each image loaded here's json!
        # level_parse = 'Level 3'
        # plot_level = 'Plot-level questions'
        # qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
        qa = []
        level_parse = question_list['Level']
        plot_level = question_list['type']
        try:
            qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
        except Exception as e:
            print('[ERROR]:', str(e))
            print('base json is:', base_json)
        # pick the right question to get the right answer
        answer = None
        for qa_pair in qa:
            if qa_pair['Q'] == question_list['Q']:
                answer = qa_pair['A']
        if answer is None:
            print("[ERROR]: question not found")
            print('qa:')
            print(qa)
            print('----------')
            print('question list:', question_list)
            print('****************')
            import sys; sys.exit()
        elif verbose:
            print('Answer:', answer)

        prompt_dummy += 'QUESTION: ' + prompt_dummy_full + '\n'
        prompt_dummy += 'ANSWER: ' + str(answer) + '\n'
        prompt_dummy += '"""\n'
        # replace text
        replace_text = f"{encoded_image}"
        prompt += 'QUESTION: ' + prompt_dummy_full.replace('[Image: <ENCODED IMAGE>]', replace_text) + '\n'
        prompt += 'ANSWER: ' + str(answer) + '\n'
        prompt += '"""\n'

    # stuff
    #prompt += '"""'
    return prompt, prompt_dummy #, answer

In [6]:
#question_list

In [7]:
# test one
fac = 1.0
question_list_test = {'Q': 'You are a helpful assistant that can analyze images.  How many gaussians were used to generate the data for the plot in the figure panel? Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.',
 'A': 1,
 'Level': 'Level 3',
 'type': 'Plot-level questions',
 'Response': 'TEST RUN',
 'persona': 'You are a helpful assistant that can analyze images.',
 'context': '',
 'question': 'How many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5.',
 'format': 'Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.',
 'plot number': 'plot0',
 'Response String': 'TEST RUN',
 'prompt': 'I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5. Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.',
 'extra sys prompt': ''
 }
prompt_sys, prompt_dummy_sys = create_fewshot(fewshot_dir, 
                                                question_list_test, 
                                                tmp_dir = tmp_dir,
                                                fac = fac, verbose=True)

MAIN QUESTION: You are a helpful assistant that can analyze images.  How many gaussians were used to generate the data for the plot in the figure panel? Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
-------- JSON: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/fewshot//jsons/id_0000.json -----------------
   on question: how many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5. Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
Answer: 2
-------- JSON: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/fewshot//jsons/id_0001.json -----------------
   on question: how many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5. Please format the output as a json as {"ngaussians":""} for this figure p

In [8]:
print(prompt_dummy_sys)

Here are some example questions with figures and their respective answers. Reference examples delimited with “”"
"""
QUESTION: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5. Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
ANSWER: 2
"""
QUESTION: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many gaussians were used to generate the data for the plot in the figure panel? Please choose an integer number from 1 to 5. Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
ANSWER: 5
"""
QUESTION: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many gaussians were used to generate the data for the 

In [9]:
def send_to_chatgpt(question_list, client, image_path, encoded_image,
                    model ="gpt-4o-mini", 
                    tmp_dir = '/Users/jnaiman/Downloads/tmp/',
                    test_run = True, fac=1.0, img_format='png',
                    verbose=True, 
                    subset_questions_by_keys = None,
                    use_fewshot = False,
                    types = ['(words + list)', '(words)']
                        ):
    # models as of ~May 2024
    #model="gpt-4",
    #model="gpt-4o",
    #model ="gpt-4o-mini",
    #model ="gpt-3.5-turbo",
    #model="gpt-3.5-turbo-instruct",
    """
    subset_questions_by_keys : only use a subset of questions to ping to model, e.g. ['median', 'how many gaussians']
    use_fewshot : if True will use some examples from the fewshot directory
    """

    iFac = 1.0 # just in case we want to progressively make the image smaller
    success = False
    is_subset = False
    prompt_dummy_sys = ''
    while not success:
        try:
            # # current question format is: ['persona', 'context','question', 'format'] (see readme in example_hists)
            # question = question_list['context'] + " " + question_list['question'] + " " + question_list['format']
            # # lowercase the first word, just in case
            # question = question.lstrip() # no whitespace
            # question = question[0].lower() + question[1:]
            # if verbose: print('   on question:',question)
            # # Prepare the API request
            # prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
            # prompt_save = f"I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, {question}"
            prompt, prompt_save, question =  make_question_and_prompt(question_list, encoded_image, verbose = verbose)

            if subset_questions_by_keys is not None and type(subset_questions_by_keys) == type([]): # make sure list
                for s in subset_questions_by_keys:
                    if s in question:
                        is_subset = True
            else:
                is_subset = True
            
            if not test_run and is_subset:
                if use_fewshot:
                    #print("HI THERE")
                    prompt_sys, prompt_dummy_sys = create_fewshot(fewshot_dir, 
                                                                 question_list, 
                                                                 tmp_dir = tmp_dir,
                                                                 fac = fac, types=types)
                else:
                    prompt_sys = ''; prompt_dummy_sys = ''
                # Send the request to the GPT-4o API
                response = client.chat.completions.create(
                    model = model,
                    messages=[
                        {"role": "system", "content": question_list['persona'] + ' ' + prompt_sys},
                        {"role":"user", "content": [
                            {
                            "type": "text",
                            "text": prompt
                            },
                            {
                            "type": "image_url",
                            "image_url": {
                                #"url": f"data:image/jpeg;base64,{encoded_image}" # PNG encoding??? does thsi need to change??
                                "url": f"data:image/{img_format};base64,{encoded_image}" 
                            }
                            }
                        ]
                        }
                    ]
                )
                success = True
            elif not is_subset:
                response = 'Not asked'
                success = True
            elif use_fewshot:
                prompt_sys, prompt_dummy_sys = create_fewshot(fewshot_dir, 
                                                                question_list, 
                                                                tmp_dir = tmp_dir,
                                                                fac = fac, types=types)
                success = True
            else:
                success = True
        except Exception as e:
            print(e)
            new_fac = fac/iFac
            print('new fac = ', new_fac)
            encoded_image = load_image(image_path,fac=new_fac, tmp_dir=tmp_dir)
            iFac += 1
            #import sys; sys.exit()
            #success = True
            # break
            klaksljf
    
    if not test_run and is_subset:
        # Get the response from the API
        answer = response.choices[0].message.content
        question_list['raw answer'] = answer
        # format answer
        answer_format = answer.replace("```json\n",'').replace("\n```",'')
        try:
            question_list['Response'] = json.loads(answer_format)
        except:
            question_list['Response'] = answer_format
            question_list['Error'] = 'JSON formatting'
        question_list['Response String'] = answer_format
        success = True
    elif not is_subset:
        answer = response
        question_list['raw answer'] = answer
        question_list['Response'] = answer
        question_list['Response String'] = answer
        success = True
    else:
        question_list['Response'] = 'TEST RUN'
        question_list['Response String'] = 'TEST RUN'

    return question_list, prompt_save, prompt_dummy_sys

In [10]:
def print_qa(pickle_file, qa_in, subset_questions_by_keys=None, showNotAsked=False):
    if subset_questions_by_keys is not None:
        print('---------- ASKED ----------')
    print(pickle_file)
    print('*********')
    for qa_pairs in qa_in:
        hasSub = False
        if subset_questions_by_keys is not None and type(subset_questions_by_keys) == type([]):
            for s in subset_questions_by_keys:
                if s in qa_pairs['prompt']:
                    hasSub = True
        else:
            hasSub = True

        if hasSub:
            print('Prompt:', qa_pairs['prompt'])
            print('   Real A:', qa_pairs['A'])
            print('ChatGPT A:', qa_pairs['Response'])
            print('')

    if subset_questions_by_keys is not None and showNotAsked:
        print('')
        print('')
        print('------------ NOT ASKED -----------')
        for qa_pairs in qa_in:
            hasSub = False
            if subset_questions_by_keys is not None and type(subset_questions_by_keys) == type([]):
                for s in subset_questions_by_keys:
                    if s in qa_pairs['prompt']:
                        hasSub = True

            if not hasSub:
                print('Prompt:', qa_pairs['prompt'])
                print('   Real A:', qa_pairs['A'])
                print('ChatGPT A:', qa_pairs['Response'])
                print('')

In [11]:
# setup
with open(key_file,'r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

In [12]:
jsons_to_parse = glob(jsons_dir + '/*.json')
jsons_to_parse[:3]

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0077.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0020.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0036.json']

Look at a possible questions:

In [14]:
json_path = jsons_to_parse[0]
verbose = False
restart = False

fac = 0.5

img_path = imgs_dir + json_path.split('/')[-1].removesuffix('.json') + '.' + img_format
encoded_image, img_format_media, base_json, err = get_img_json_pair(img_path, json_path, None, 
                                                    fac=fac, restart=restart,
                                                    tmp_dir=tmp_dir)

base_json['VQA']

{'Level 1': {'Figure-level questions': {},
  'Plot-level questions': {'nbars': {'plot0': {'Q': 'You are a helpful assistant that can analyze images.  How many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.',
     'A': {'nbars': 20},
     'persona': 'You are a helpful assistant that can analyze images.',
     'context': '',
     'question': 'How many bars are there in the specified figure panel?',
     'format': 'Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.'}}}},
 'Level 2': {'Plot-level questions': {'minimum': {'plot0': {'Q': 'You are a helpful assistant that can analyze images.  What are the minimum data values  in this figure panel?  Please format the output as a json as {"minimum x":""} for this figure panel, where the "minimum x" value should be a float, calculated from the data values used 

In [15]:
#model = "gpt-5-nano-2025-08-07"
#subset_questions_by_keys = ['median', 'ngaussians'] # set to None to do all questions

#p, pf, response = create_fewshot(fewshot_dir, dir_api, question_list, model, subset_questions_by_keys)

In [16]:
#print(pf)

In [ ]:
iMax = 150 # should be 80
verbose = False
test_run = False # run w/o actually pinging openai
restart = False
print_sysprompt = False
use_fewshot = True
#model ="gpt-4o-mini"
model = "gpt-5-nano-2025-08-07"

subset_questions_by_keys = ['median', 'ngaussians'] # set to None to do all questions

fac = 0.5
for ijson,json_path in enumerate(jsons_to_parse):
    if ijson >= iMax:
        continue

    print('on', ijson+1, 'of', min(iMax,len(jsons_to_parse)), ':', json_path.split('/')[-1])

    # get image and base json
    img_path = imgs_dir + json_path.split('/')[-1].removesuffix('.json') + '.' + img_format
    encoded_image, img_format_media, base_json, err = get_img_json_pair(img_path, json_path, dir_api, 
                                                      fac=fac, restart=restart,
                                                      tmp_dir=tmp_dir)

    if err:
        continue


    ###### create QA ########
    qa = []
    
    for k,v in base_json['VQA']['Level 1']['Figure-level questions'].items():
        out = {'Q':v['Q'], 'A':v['A'], 'Level':'Level 1', 'type':'Figure-level questions', 'Response':""}
        qa.append(out)
    
    # what kinds?
    types = ['(words + list)', '(words)']
    
    # get uniques
    level_parse = 'Level 1'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
    
    level_parse = 'Level 2'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)
    
    level_parse = 'Level 3'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, base_json['VQA'], types)

    responses = []
    for question_list in qa:
        response, prompt, prompt_sys = send_to_chatgpt(question_list, client, img_path, encoded_image,
                    model = model, img_format = img_format_media,
                    test_run = test_run, subset_questions_by_keys=subset_questions_by_keys, 
                    types=types, use_fewshot=use_fewshot)
        responses.append(response)
        question_list['prompt'] = prompt
        question_list['extra sys prompt'] = prompt_sys
        if print_sysprompt:
            print('       System prompt:', prompt_sys)

    # parse for errors
    qa = parse_for_errors(qa)

    # dump to file
    if not test_run:
        with open(dir_api + json_path.split('/')[-1].removesuffix('.json')+ '.pickle', 'wb') as ff:
            pickle.dump([qa, model], ff)
        print("just saved:", dir_api + json_path.split('/')[-1].removesuffix('.json')+ '.pickle')
    else:
        print('Would store at:', dir_api + json_path.split('/')[-1].removesuffix('.json')+ '.pickle')


on 1 of 80 : id_0077.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0077.pickle
on 2 of 80 : id_0020.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0020.pickle
on 3 of 80 : id_0036.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0036.pickle
on 4 of 80 : id_0061.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0061.pickle
on 5 of 80 : id_0016.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0016.pickle
on 6 of 80 : id_0041.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0041.pickle
on 7 of 80 : id_0057.json
have file already: /Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api_fewshot/id_0057.pickle
on 8 of 80 : id_0000.json
have fil

In [190]:
jsons_to_parse

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0077.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0020.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0036.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0061.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0016.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0041.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0057.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0000.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0001.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0056.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0040.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0017.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0060.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/id_0037.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/

In [12]:
qa

[{'Q': 'You are a helpful assistant that can analyze images.  How many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.',
  'A': 60,
  'Level': 'Level 1',
  'type': 'Plot-level questions',
  'Response': 'Not asked',
  'persona': 'You are a helpful assistant that can analyze images.',
  'context': '',
  'question': 'How many bars are there in the specified figure panel?',
  'format': 'Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.',
  'plot number': 'plot0',
  'raw answer': 'Not asked',
  'Response String': 'Not asked',
  'prompt': 'I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.'},
 {'Q': 'You

## Look at data

Check out one, if you wanna:

In [40]:
pickles = glob(dir_api + '*.pickle')
pickles[:5]

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0039.pickle',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0044.pickle',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0027.pickle',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0056.pickle',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0035.pickle']

In [ ]:
ifile = 0
with open(pickles[ifile], 'rb') as f:
    qa_in = pickle.load(f)[0]

In [42]:
print_qa(pickles[ifile], qa_in, subset_questions_by_keys=subset_questions_by_keys, showNotAsked=False)

---------- ASKED ----------
/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/chatgpt_api/id_0039.pickle
*********
Prompt: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, what are the median data values  in this figure panel?  Please format the output as a json as {"median x":""} for this figure panel, where the "median x" value should be a float, calculated from the data values used to create the plot.
   Real A: 0.2655385491960254
ChatGPT A: {'median x': 0.27}

Prompt: I am going to show you an image. Here is the image: [Image: <ENCODED IMAGE>]. Now, how many gaussians were used to generate the data for the plot in the figure panel? Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
   Real A: 3
ChatGPT A: {'ngaussians': 3}

